In [ ]:
!pip install VIM
!pip install naniar
!pip install missMDA
!pip install Amelia
!pip install mice
!pip install missForest
!pip install FactoMineR
!pip install Tidyverse

# Python code equivalent of R code
# Note: Some packages may need to be installed via pip or conda

import pandas as pd
import numpy as np
import re
from VIM import impute
from naniar import miss_var_summary, miss_case_summary
from missMDA import imputePCA
from Amelia import amelia
from mice import MICE
from missForest import missForest
from FactoMineR import PCA
from Tidyverse import *


## Imputation Technique ##

In [ ]:


# data = pd.read_csv("~/Downloads/Chalice Family Dollar Q4 2022 (Raw) (8).csv")
# data = pd.read_csv("~/Downloads/Chalice Family Dollar Q4 2022 (Raw).csv")

# convert data to a pandas DataFrame
data = pd.DataFrame(data)
data = data.apply(lambda x: pd.to_numeric(x, errors="coerce"), axis=0) # convert all columns to numeric type

# custom = pd.DataFrame(data.iloc[:, 16])
# custom = pd.DataFrame(data.iloc[:, 18])
custom = pd.DataFrame(data.iloc[:, 12])

# custom = pd.DataFrame(data.iloc[:, 16])

custom = custom.str.split(",", expand=True)
custom.columns = ["c1", "c2", "c3", "c4", "c5"]

custom[["var", "c1"]] = custom["c1"].str.split(":", expand=True)
custom["c1"] = custom["c1"].apply(lambda x: re.sub('[[:punct:] ]+', ' ', x))

custom[["var", "c2"]] = custom["c2"].str.split(":", expand=True)
custom["c2"] = custom["c2"].apply(lambda x: re.sub('[[:punct:] ]+', ' ', x))

custom[["var", "c3"]] = custom["c3"].str.split(":", expand=True)
custom["c3"] = custom["c3"].apply(lambda x: re.sub('[[:punct:] ]+', ' ', x))

custom[["var", "c4"]] = custom["c4"].str.split(":", expand=True)
custom["c4"] = custom["c4"].apply(lambda x: re.sub('[[:punct:] ]+', ' ', x))

custom[["var", "c5"]] = custom["c5"].str.split(":", expand=True)
custom["c5"] = custom["c5"].apply(lambda x: re.sub('[[:punct:] ]+', ' ', x))

custom = custom.drop(columns=["var"])
data = pd.concat([data, custom], axis=1)

# Predictive mean matching (multiple imputation)
data = data.replace("", np.nan)

# df = data.iloc[:, 14:23].join(data.iloc[:, 86:90])
# df = data.iloc[:, 18:28].join(data.iloc[:, 86:90])
# df = data.iloc[:, 20:30].join(data.iloc[:, 76:80])
df = data.iloc[:, 14:23]

##yaml, "column names for the change in the config file"
##test out in the data bricks (hold out with cv = 6)

final = data.drop(columns=df.columns)

non_miss = df.notna().sum(axis=1)
m = missForest(df.values, ntree=100, parallel=True, variablewise=False, maxiter=10, minprop=0.1)
imputed_df = pd.DataFrame(m["ximp"])
imputed_df.columns = df.columns
final


## Brand Lift Code ##

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import geopandas as gpd
import mapclassify
from mapclassify import Quantiles, UserDefined, NaturalBreaks, EqualInterval
import mapview
import folium
from folium.plugins import HeatMap
import rasterio
import rasterstats
from rasterstats import zonal_stats
import os
import requests
from zipfile import ZipFile
from io import BytesIO
import datetime
import string
import re
import openpyxl
import likert
from IPython.display import display, HTML


In [ ]:

data = pd.read_csv("final.csv")

creative = data[['Creative.Name']]
creative = creative['Creative.Name'].str.split(pat='-', n=-1, expand=True)
creative = creative.iloc[:, [1, 2]]
creative.columns = ["creativename", "creativeduration"]
creative['creativeduration'][creative['creativeduration'].str.contains('15')] = "15s"
creative['creativeduration'][creative['creativeduration'].str.contains('6')] = "6s"
creative['creativeduration'][creative['creativeduration'].str.contains('30')] = "30s"

data = pd.concat([data, creative], axis=1)

ZipCodeSourceFile = "http://download.geonames.org/export/zip/US.zip"
response = requests.get(ZipCodeSourceFile)
ZipCodes = pd.read_csv(BytesIO(response.content), sep="\t", header=None)
ZipCodes = ZipCodes.iloc[:, [1, 2, 3, 4, 5, 9, 10, 11]]

df = pd.merge(data, ZipCodes, how='left', left_on='Demo.ZIP', right_on=1)
df = df.drop(1, axis=1)

df['date'] = pd.to_datetime(df['Response.Date'], format="%Y-%m-%d")
df[['date', 'timezone']] = df['date'].astype(str).str.split(" ", expand=True)
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")

df['week'] = df['date'].dt.strftime('%V').astype(int)
df['week'] = np.where(df['week'] == 1, 9, df['week'])
df['week'] = np.where(df['week'] == 2, 10, df['week'])
df['week'] = np.where(df['week'] == 3, 11, df['week'])
df['week'] = np.where(df['week'] == 4, 12, df['week'])
df['week'] = np.where(df['week'] == 5, 13, df['week'])

df['week'] = np.where(df['week'] == 44, 1, df['week'])
df['week'] = np.where(df['week'] == 45, 2, df['week'])
df['week'] = np.where(df['week'] == 46, 3, df['week'])
df['week'] = np.where(df['week'] == 47, 4, df['week'])
df['week'] = np.where(df['week'] == 48, 5, df['week'])
df['week'] = np.where(df['week'] == 49, 6, df['week'])
df['week'] = np.where(df['week'] == 50, 7, df['week'])


In [ ]:
df['week'] = pd.to_numeric(df['week'])
df = df.sort_values(by=['week'])

week_names = {1: '11/02/22', 2: '11/07/22', 3: '11/14/22', 4: '11/21/22', 5: '11/28/22', 
              6: '12/05/22', 7: '12/12/22', 8: '12/19/22', 9: '01/05/23', 10: '01/09/23', 
              11: '01/16/23', 12: '01/23/23', 13: '01/28/23'}
df['Weekname'] = df['week'].map(week_names)

df = df[df['date'] >= '2023-01-05']

df['Demo.Age'] = pd.to_numeric(df['Demo.Age'])
age_bins = [0, 18, 24, 34, 44, 54, 64, float('inf')]
age_labels = ['0-18', '19-24', '25-34', '35-44', '45-54', '55-64', '65+']
df['Age Group'] = pd.cut(df['Demo.Age'], bins=age_bins, labels=age_labels, include_lowest=True)

df = df.sort_values(by=['Demo.State', 'Age Group', 'week'])

In [ ]:
from dplython import *

brand = df[df['Family.Dollar..Brand.Awareness'] == '2']


m = (DplyFrame(brand)
     .filter_by(Group="Exposed")
     .group_by('week', 'Demo.State', 'Strata')
     .mutate(Total_exposed=X.Weight.sum())
     .ungroup()
     .group_by('week', 'Demo.State')
     .mutate(Total_exposed_strata=X.Total_exposed.sum())
     .ungroup())

n = (DplyFrame(m)
     .group_by('Demo.State')
     .mutate(prop=(X.Total_exposed_strata/X.Total_exposed_strata.sum())*100))

k = (DplyFrame(brand)
     .filter_by(Group="Control")
     .group_by('week', 'Demo.State', 'Strata')
     .mutate(Total_control=X.Weight.sum())
     .ungroup()
     .group_by('week', 'Demo.State')
     .mutate(Total_control_strata=X.Total_control.sum())
     .ungroup())

l = (DplyFrame(k)
     .group_by('Demo.State')
     .mutate(prop=(X.Total_control_strata/X.Total_control_strata.sum())*100))



In [ ]:

brand_lift = pd.concat([n, l], ignore_index=True)

brand_lift = (DplyFrame(brand_lift)
              .group_by('Demo.State', 'week')
              .mutate(lift=(X.prop - X.prop.shift(fill_value=0)))
              .mutate(lift=X.lift.round(2)))

brand_lift.loc[brand_lift['lift'].isna(), 'lift'] = 0

brand_lift['lift'] = pd.to_numeric(brand_lift['lift'])

In [ ]:

import pivottablejs as pt

# group by c5, arrange by week, and calculate cumulative lift
brand_lift = brand_lift.sort_values(by=['week']).groupby(['c5']).apply(
    lambda x: x.assign(
        increment=pd.Series([x['lift'].iloc[0]] + list(np.maximum(np.diff(x['lift']), 0))),
        Cumulative=lambda y: np.cumsum(y['increment']),
        cumulative=lambda z: (z['Cumulative'] - np.mean(z['Cumulative'])) / np.std(z['Cumulative'])
    )
).reset_index(drop=True)

# # filter by date
# brand_lift_m = brand_lift[brand_lift['date'] >= '2023-01-05']
# brand_lift_all = df[df['date'] >= '2023-01-05']

# create heatmap using pivottablejs
pt.pivot_ui(brand_lift, rows=['c5'], cols=['Weekname', 'week'], 
            vals='cumulative', aggregatorName='Last', rendererName='Heatmap', 
            colOrder="key_a_to_z", rowOrder="key_a_to_z", 
            rendererOptions={
                'heatmap': {
                    'colorScaleGenerator': lambda values: pt.color_scale(values, 'green')
                }
            })


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# create table
table = pd.pivot_table(brand_lift_m, values='cumulative', index='creativename', columns='date')

# calculate cumulative lift and maximum value per week
d4 = brand_lift.groupby(['Weekname', 'date']).agg({'cumulative': 'last'}).reset_index()
d5 = d4.groupby('Weekname').agg({'cumulative': 'max'}).reset_index()[1:]
d5['Weekname'] = pd.to_datetime(d5['Weekname'], format='%m/%d/%y')
d5['Max'] = round(d5['Max'], digits=2)



In [ ]:
# create line plot
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(data=d5, x='Weekname', y='Max', color='forestgreen')
sns.scatterplot(data=d5, x='Weekname', y='Max', color='darkgreen')
sns.despine(left=True)
ax.set(title='Family Dollar Brand Lift', xlabel='Weeks', ylabel='Cumulative %')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%y'))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))



In [ ]:
old = brand_lift.loc[brand_lift['date'] < pd.to_datetime("2022-11-28")]
new = brand_lift.loc[brand_lift['date'] >= pd.to_datetime("2022-11-28")]

old_data = old.groupby(['Weekname', 'date'])['cumulative'].last().reset_index()

old_data = old_data.loc[old_data['date'] >= pd.to_datetime("2022-10-31")]

new_data = new.groupby(['Weekname', 'date'])['cumulative'].last().reset_index()

old_data['type'] = 'Before Algo Push'
new_data['type'] = '11/28: After Algo Push'

d4 = pd.concat([old_data, new_data])

sns.barplot(x='type', y='cumulative', data=d4, hue='type', palette=['forestgreen', 'red'])
plt.title("Family Dollar Brand Lift")
plt.xlabel("")
plt.ylabel("Cumulative %")
plt.legend([],[], frameon=False)
plt.show()